# Optical spectra from casida
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/response/optical_spectra_from_casida/)

In this tutorial we will again calculate the absorption spectrum of methane, but this time using Casida's equations.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from postopus import Run

In [ ]:
!mkdir -p 3_Optical_spectra_from_casida

In [ ]:
cd 3_Optical_spectra_from_casida

## Ground-state

Once again our first step will be the calculation of the ground state. We will use the following input file:


In [ ]:
%%writefile inp

stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

CalculationMode = gs
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

In [ ]:
!octopus

Note that we are using the values for the spacing and radius that were found in the [Convergence_of_the_optical_spectra tutorial](2_Convergence_of_optical_spectra.ipynb) to converge the absorption spectrum.

## Unoccupied States

The Casida equation is a (pseudo-)eigenvalue equation written in the basis of particle-hole states. This means that we need both the occupied states -- computed in the ground-state calculation -- as well as the unoccupied states, that we will now obtain, via a non-self-consistent calculation using the density computed in `gs`. The input file we will use is


In [ ]:
%%writefile inp

stdout = 'stdout_unocc.txt'
stderr = 'stderr_unocc.txt'

CalculationMode = unocc
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

ExtraStates = 10

Here we have changed the [CalculationMode](https://www.octopus-code.org/documentation//13/variables/calculation_modes/calculationmode) to `gs` and added 10 extra states by setting the [ExtraStates](https://www.octopus-code.org/documentation//13/variables/states/extrastates) input variable.

By running **Octopus**, we will obtain the first 10 unoccupied states. The solution of the unoccupied states is controlled by the variables in section [Eigensolver](https://www.octopus-code.org/documentation//13/variables/scf/eigensolver/eigensolver). You can take a look at the eigenvalues of the unoccupied states in the file `static/eigenvalues`:

In [ ]:
!octopus

In [ ]:
run = Run(".")
run.default.scf.eigenvalues()

##  Casida calculation

Now modify the [CalculationMode](https://www.octopus-code.org/documentation//13/variables/calculation_modes/calculationmode) to `gs` and rerun **Octopus**. Note that by default **Octopus** will use all occupied and unoccupied states that it has available.

Sometimes, it is useful not to use all states. For example, if you have a molecule with 200 atoms and 400 occupied states ranging from -50 to -2 eV, and you are interested in looking at excitations in the visible, you can try to use only the states that are within 10 eV from the Fermi energy. You could select the states to use with [CasidaKohnShamStates](https://www.octopus-code.org/documentation//13/variables/linear_response/casida/casidakohnshamstates) or [CasidaKSEnergyWindow](https://www.octopus-code.org/documentation//13/variables/linear_response/casida/casidaksenergywindow).



In [ ]:
%%writefile inp

stdout = 'stdout_casida.txt'
stderr = 'stderr_casida.txt'

CalculationMode = casida
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

ExtraStates = 10

In [ ]:
!octopus

A new directory will appear named `casida`, where you can find the file `casida/casida`:


In [ ]:
!cat casida/casida

The &lt;x&gt;, &lt;y&gt; and &lt;z&gt; are the transition dipole moments:

$$
  \langle x \rangle = \langle \Phi\_0|x|\Phi_I \rangle
  \,;\qquad
  \langle y \rangle = \langle \Phi\_0|y|\Phi_I \rangle
  \,;\qquad
  \langle z \rangle = \langle \Phi\_0|z|\Phi_I \rangle
$$

where $\Phi_0$ is the ground state and $\Phi_I$ is the given excitation. The
oscillator strength is given by:

$$
  f_I = \frac{2 m_e}{3 \hbar^2} \omega_I \sum_{n\in x,y,z} |\langle\Phi\_0|n|\Phi_I \rangle |^2
$$

as the average over the three directions. The optical absorption spectrum can be given as the "strength function",
which is

$$
  S(\omega) = \sum_I f_I \delta(\omega-\omega_I)\\,
$$

Note that the excitations are degenerate with degeneracy 3. This could already be expected from the $T_d$ symmetry of methane.

{{% notice note %}}
Note that within the degenerate subspaces, there is some arbitrariness (possibly dependent on the details of your compilation and machine) in the linear combinations of transitions. Therefore, you should not be concerned if you do not have the same results for the components of the transition dipole moments (above) and analysis of the excitations (below). However, the energies and resulting spectra should agree.
{{% /notice %}}

Further information concerning the excitations can be found in the directory `casida/casida_excitations`. For example, the first excitation at 9.27 eV is analyzed in the file `casida/casida_excitations/00001`:


In [ ]:
!cat casida/casida_excitations/00001

These files contain basically the eigenvector of the Casida equation. The first two columns are respectively the index of the occupied and the index of the unoccupied state, the third is the spin index (always 1 when spin-unpolarized), and the fourth is the coefficient of that state in the Casida eigenvector. This eigenvector is normalized to one, so in this case one can say that 74.7% (0.864<sup>2</sup>) of the excitation is from state 3 to state 5 (one of 3 HOMO orbitals->LUMO) with small contribution from some other transitions.

##  Absorption spectrum

To visualize the spectrum, we need to broaden these delta functions with the utility `oct-casida_spectrum` (run in your working directory, not in `casida`). It convolves the delta functions with Lorentzian functions. The operation of `oct-casida_spectrum` is controlled by the variables [CasidaSpectrumBroadening](https://www.octopus-code.org/documentation//13/variables/utilities/oct-casida_spectrum/casidaspectrumbroadening) (the width of this Lorentzian), [CasidaSpectrumEnergyStep](https://www.octopus-code.org/documentation//13/variables/utilities/oct-casida_spectrum/casidaspectrumenergystep), [CasidaSpectrumMinenergy](https://www.octopus-code.org/documentation//13/variables/utilities/oct-casida_spectrum/casidaspectrumminenergy), and [CasidaSpectrumMaxEnergy](https://www.octopus-code.org/documentation//13/variables/utilities/oct-casida_spectrum/casidaspectrummaxenergy). If you run  `oct-casida_spectrum` you obtain the file `casida/spectrum.casida`. It contains all columns of `casida` broadened.


Currently oct-utilities are not supported by postopus. This feature will be implemented soon.

In [ ]:
!oct-casida_spectrum

In [ ]:
%%writefile inp

stdout = 'stdout_td.txt'
stderr = 'stderr_td.txt'

CalculationMode = td
FromScratch = yes
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

TDPropagator = aetrs
TDTimeStep = 0.0023/eV
TDMaxSteps = 4350  # ~ 10.0/TDTimeStep

TDDeltaStrength = 0.01/angstrom
TDPolarizationDirection = 1

The time required to run the following cell is in the order of 5 to 10 minutes.

In [ ]:
!octopus

In [ ]:
!oct-propagation_spectrum

In [ ]:
df_casida = pd.read_csv(
    "casida/spectrum.casida",
    delimiter="  ",
    usecols=[0, 4],
    names=["Energy [eV]", "StrengthFunction"],
    skiprows=1,
    engine="python",
)

df_time_propagation = pd.read_csv(
    "cross_section_vector",
    delimiter="  ",
    usecols=[0, 3],
    names=["Energy [eV]", "StrengthFunction"],
    skiprows=26,
    engine="python",
)

fig, ax = plt.subplots()
ax.set_title("Absorption spectrum")
ax.set_xlabel("Energy (eV)")
ax.set_ylabel("Strength function (1/eV)")

df_casida.plot(
    x="Energy [eV]",
    y="StrengthFunction",
    ax=ax,
    label="casida",
    color="green",
    xlim=[0, 20],
)

df_time_propagation.plot(
    x="Energy [eV]",
    y="StrengthFunction",
    ax=ax,
    label="time-propagation",
    color="purple",
    xlim=[0, 20],
);

Comparing the spectrum obtained with the time-propagation in the [Convergence of the optical spectra tutorial](2_Convergence_of_optical_spectra.ipynb) with the one obtained with the Casida approach using the same grid parameters, we can see that

* The peaks of the time-propagation are broader. This can be solved by either increasing the total propagation time, or by increasing the broadening in the Casida approach.
* The first two peaks are nearly the same. Probably also the third is OK, but the low resolution of the time-propagation does not allow to distinguish the two close peaks that compose it.
* For high energies the spectra differ a lot. The reason is that we only used 10 empty states in the Casida approach. In order to describe better this region of the spectrum we would need more. This is why one should always check the convergence of relevant peaks with respect to the number of empty states.

You probably noticed that the Casida calculation took much less time than the time-propagation. This is clearly true for small or medium-sized systems. However, the implementation of Casida in **Octopus** has a much worse scaling with the size of the system than the time-propagation, so for larger systems the situation may be different. Note also that in Casida one needs a fair amount of unoccupied states which are fairly difficult to obtain.

If you are interested, you may also compare the Casida results against the Kohn-Sham eigenvalue differences in `casida/spectrum.eps_diff` and the Petersilka approximation to Casida in `casida/spectrum.petersilka`.


[Go to *4_Optical_spectra_from_Sternheimer.ipynb*](4_Optical_spectra_from_Sternheimer.ipynb)